In [16]:
import pandas as pd

In [99]:
reviews = io.load('preproc_reviews')
reviews.head()

2019-06-14 20:21:59,822 - kedro.io.data_catalog - INFO - Loading data from `preproc_reviews` (PickleLocalDataSet)...


,product_id,author_age,review_title,review_text,star_rating,recommend_flag,upvotes,product_category_division,product_category_department,product_category_class
id,,,,,,,,,,
0,767,33,NaN,Absolutely wonderful - silky and sexy and comf...,4,1,0,intimates,intimate,intimates
1,1080,34,NaN,Love this dress! it's sooo pretty. i happene...,5,1,4,general,dresses,dresses
2,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,3,0,0,general,dresses,dresses
3,1049,50,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",5,1,0,general petite,bottoms,pants
4,847,47,Flattering shirt,This shirt is very flattering to all due to th...,5,1,6,general,tops,blouses


In [96]:
def fix_missing_values(reviews: pd.DataFrame) -> pd.DataFrame:
    """Remove or impute missing values.
        
        Args:
            reviews: source data.
        Returns:
            Primary dataframe with missing values removed or imputed.
    
    """
    
    reviews = reviews.dropna(subset = ['review_text'])
    
    return reviews

In [97]:
reviews = fix_missing_values(reviews)
reviews.isnull().sum()

product_id                        0
author_age                        0
review_title                   2966
review_text                       0
star_rating                       0
recommend_flag                    0
upvotes                           0
product_category_division        13
product_category_department      13
product_category_class           13
dtype: int64

In [36]:
products_without_category = reviews\
    [reviews.\
         filter(like = 'category')\
         .isnull()\
         .any(1)]\
    .product_id\
    .unique()
products_without_category

array([ 72, 492, 152, 184, 772, 665, 136])

In [52]:
reviews[reviews.product_id.isin(products_without_category)][['product_id', 'review_title', 'review_text']]

,product_id,review_title,review_text
id,,,
9444,72,My favorite socks!!!,"I never write reviews, but these socks are so ..."
13767,492,So soft!,I just love this hoodie! it is so soft and com...
13768,492,Wardrobe staple,Love this hoodie. so soft and goes with everyt...
16216,152,Warm and cozy,"Just what i was looking for. soft, cozy and warm."
16221,152,Love!,I am loving these. they are quite long but are...
16223,152,"""long and warm""",These leg warmers are perfect for me. they are...
18626,184,Nubby footless tights,"These are amazing quality. i agree, size up to..."
18671,184,New workhorse,These tights are amazing! if i care for them w...
20088,772,Comfy sweatshirt!,This sweatshirt is really nice! it's oversize...


In [94]:
imputed_product_types = {
    72: 'socks',
    492: 'hoodie',
    152: 'leg warmer',
    184: 'tights',
    772: 'sweatshirt',
    665: None,
    136: 'socks'
}

In [95]:
reviews[reviews\
            .review_title\
            .str.contains(imputed_product_types[772], na = False)]\
        .filter(like = 'category')\
        .drop_duplicates()

,product_category_division,product_category_department,product_category_class
id,,,
283,general,tops,knits
363,intimates,intimate,lounge
3689,general,jackets,jackets
7303,general,dresses,dresses
10997,intimates,intimate,sleep
14483,general petite,tops,knits
20088,NaN,NaN,NaN


In [53]:
def print_full(x):
    pd.set_option('display.max_rows', len(x))
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', 2000)
    pd.set_option('display.float_format', '{:20,.2f}'.format)
    pd.set_option('display.max_colwidth', -1)
    print(x)
    pd.reset_option('display.max_rows')
    pd.reset_option('display.max_columns')
    pd.reset_option('display.width')
    pd.reset_option('display.float_format')
    pd.reset_option('display.max_colwidth')

In [87]:
reviews.loc[2747].filter(like = 'review')

review_title                                  Unique little socks
review_text     Unique pattern. got lots of compliments! great...
Name: 2747, dtype: object

In [55]:
category_hierarchy = ['product_category_division', 'product_category_department', 'product_category_class']
reviews\
    .filter(category_hierarchy)\
    .drop_duplicates()\
    .sort_values(category_hierarchy)

,product_category_division,product_category_department,product_category_class
id,,,
12397,general,bottoms,casual bottoms
224,general,bottoms,jeans
15,general,bottoms,pants
638,general,bottoms,shorts
27,general,bottoms,skirts
1,general,dresses,dresses
121,general,jackets,jackets
18,general,jackets,outerwear
4,general,tops,blouses
